
# Example: Bimatrix Game

This notebook provides example code to make it easier to write a player function. 

* Write player functions, `player1.py`, `player2.py`, ... in the folder `./players/`. 
* Run this notebook: it will automatically read them all.

In [1]:
from game_tournament.game import DiscreteGame, Tournament
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

%load_ext autoreload
%autoreload 2 

 Convenient function for printing payoffs in bimatrix games 

In [2]:
def print_payoffs(U1, U2, A1, A2): 
    '''print_payoffs: 
    Args 
        U1, U2: (na1,na2) arrays of payoffs 
        A1: na1 list of strings: labels for actions for player 1 
        A2: na2 list of strings: labels for actions for player 2
    '''
    na1,na2 = U1.shape 

    # "matrix" of tuples 
    X = [[(U1[r,c],U2[r,c]) for c in range(na2)] for r in range(na1)]

    # dataframe version 
    tab = pd.DataFrame(X, columns=A2, index=A1)

    return tab 

# Reading in player modules 

In [3]:
# path to where player functions should be located for this game 
player_path = './players/'

# A Bimatrix Game

`U1` and `U2` are the payoff matrices for the bimatrix game that we want to work with in this notebook. 

In [4]:
np.random.seed(1337)
U1 = np.array([[5, 3, 1], [3, 2, 3], [2, 1, 0]])
U2 = np.array([[0, 3, 1], [4, 2, 1], [2, 1, 5]])

A1 = ["U", "M", "D"]
A2 = ["L", "C", "R"]

print_payoffs(U1, U2, A1, A2)

,L,C,R
U,"(5, 0)","(3, 3)","(1, 1)"
M,"(3, 4)","(2, 2)","(3, 1)"
D,"(2, 2)","(1, 1)","(0, 5)"


# A single game

We will first set up a single game and play it between two directly imported player functions.

In [5]:
# let us import the two player functions, saved in the folder './players_discrete/' 
# with the filenames 'player1.py' and 'player2.py'. 
from players.player1 import player as player1
from players.player2 import player as player2
print(f'Player 1: {player1.name}')
print(f'Player 2: {player2.name}')

Player 1: 1st order
Player 2: 2nd order


In [6]:
# initialize instances of two player functions
# (which we imported earlier)
p1 = player1()
p2 = player2()

G = DiscreteGame(p1, p2, U1, U2)
print(G)

1st order vs. 2nd order: played 0 rounds


In [7]:
# before the game is played, the history is empty 
G.history

array([], shape=(0, 2), dtype=int64)

In [8]:
T = 2 # the number of rounds played of the game (repetitions of the static game to eliminate simulation noise)
G.play_game(T) # this populates the "history" property 
G.history # now history is (N,2): a row per round and column per player

array([[0, 1],
       [0, 1]])

In [9]:
G.get_game_actions_and_payoffs() # we can use a function to print the history in a pandas dataframe

actions             payoffs          
      1st order 2nd order 1st order 2nd order
Round                                        
0             0         1       3.0       3.0
1             0         1       3.0       3.0

In [10]:
G.score_game()

array([3., 3.])

In [11]:
print(G) 

1st order vs. 2nd order: played 2 rounds


# Running a tournament 

By default, the `Tournament` object will read all the `.py` files located in the folder `player_path` and run an "all-play-all" tournament between all combinations of players. Inside the tournament instance, `t`, it will store 
* `t.games`: a list of all the games that have been played. 

Each game is exactly as above, so e.g. it has a history, `t.games[0].history`, and we can use the same functions as above, e.g. `t.games[0].get_game_actions_and_payoffs()`. 

In [12]:
T = 100 # we can repeat each matchup many times to smooth out any randomness from player functions
game_data = {'U1':U1, 'U2':U2}
t = Tournament(player_path, DiscreteGame, game_data=game_data, T=T, tournament_name='A 3x3 Game')
t.run() # run the tournament

3it [00:00, 70.42it/s]

Tournament winner was: 1st order (against 2 opponents)


,A 3x3 Game
Player,
1st order,3.020
2nd order,2.445
Randawg,1.590


In [13]:
# take a look at the individual matchups of player functions 
match_results = t.get_matchup_results()
match_results

Opponent,1st order,Randawg,2nd order
Player,,,
1st order,NaN,3.04,3.00
Randawg,1.29,NaN,1.89
2nd order,3.00,1.89,NaN


In [14]:
t.scoreboard()

,A 3x3 Game
Player,
1st order,3.020
2nd order,2.445
Randawg,1.590


## Exploring individual games of the tournament

We can access the individual games in the tournament afterwards, stored as the list `tournament.games`. For debugging purposes, it may be useful to access the play to verify that your function is doing what you expect it to. 

In [15]:
print(f'Matchups in tournament')
for i,g in enumerate(t.games): 
    print(f'{i}: {g.name}')

i = 0 # <--- choose a game here 
g = t.games[0]
print(f'Printing game info for {i=}: {g.name}')
g.get_game_actions_and_payoffs()

Matchups in tournament
0: 1st order vs. Randawg
1: 1st order vs. 2nd order
2: Randawg vs. 2nd order
Printing game info for i=0: 1st order vs. Randawg


actions           payoffs        
      1st order Randawg 1st order Randawg
Round                                    
0             0       1       3.0     3.0
1             0       0       5.0     0.0
2             0       0       5.0     0.0
3             0       1       3.0     3.0
4             0       2       1.0     1.0
...         ...     ...       ...     ...
95            0       2       1.0     1.0
96            0       2       1.0     1.0
97            0       2       1.0     1.0
98            0       1       3.0     3.0
99            0       2       1.0     1.0

[100 rows x 4 columns]

# Next step: Other games

Make sure your code is able to handle different types of games. Below are a few examples, but you can also think about other games that you have encountered thus far in the course. 

In [20]:
U1 = np.array([[2,0],[0,1]])
U2 = np.array([[1,0],[0,2]])
print_payoffs(U1, U2,A1=['Bach','Stravinsky'],A2=['Bach','Stravinsky'])
t = Tournament(player_path, DiscreteGame, game_data={'U1':U1, 'U2':U2}, T=T, tournament_name='A 2x2 Game')
t.run()

3it [00:00, 37.54it/s]

Tournament winner was: 1st order (against 2 opponents)


,A 2x2 Game
Player,
1st order,1.46
2nd order,0.76
Randawg,0.75


In [21]:
U1 = np.array([[1,1],[1,5]])
U2 = np.array([[3,0],[1,5]])
print_payoffs(U1, U2,A1=['T','D'],A2=['L','R'])
t = Tournament(player_path, DiscreteGame, game_data={'U1':U1, 'U2':U2}, T=T, tournament_name='A 2x2 Game')
t.run()

3it [00:00, 77.39it/s]

Tournament winner was: 1st order (against 2 opponents)


,A 2x2 Game
Player,
1st order,3.9
2nd order,3.5
Randawg,2.7


In [27]:
T=100
U1 = np.array([[0,1,-1],[-1,0,1],[1,-1,0]])
U2 = -U1 # i.e. it's a zero sum game 
print_payoffs(U1, U2,A1=['Sten','Saks','Papir'],A2=['Sten','Saks','Papir'])
t = Tournament(player_path, DiscreteGame, game_data={'U1':U1, 'U2':U2}, T=T, tournament_name='A 2x2 Game')
t.run()

3it [00:02,  1.23it/s]

Tournament winner was: 2nd order (against 2 opponents)


,A 2x2 Game
Player,
2nd order,0.49560
Randawg,0.00455
1st order,-0.50015


Spend some time thinking about why the win probabilities are not equal in rock paper scissors!